In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time

/opt/anaconda3/envs/ths_dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10) # No softmax needed due to from_logits=True in loss
    ])

In [3]:
def get_datasets():

    def preprocess(data):
        # Normalize and ensure correct type
        image = tf.cast(data['image'], tf.float32) / 255.
        return image, data['label']

    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )

    return ds_train, ds_test

In [ ]:
learning_rate = 1e-3
batch_size = 128
num_epochs = 10

train_ds, test_ds = get_datasets()
train_ds = train_ds.cache().shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)


model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
print("Starting TensorFlow/Keras training...")
start_time = time.time()

# Training using the high-level fit() function
history = model.fit(
    train_ds,
    epochs=num_epochs,
    validation_data=test_ds,
    verbose=2 # Shows one line per epoch
)

end_time = time.time()

print("-" * 30)
print(f"TensorFlow/Keras Training Time: {end_time - start_time:.4f} seconds")
print("-" * 30)